# Machine to Machine Interaction

using **HTTP** server side

## Part I

HTTP Verbs
1. POST
2. GET
3. PUT
4. DELETE
5. PATCH

Web servers are able to respond to either of the different HTTP requests.

**Example application** (web service) capable of doing the following:
1. Store some configuration data (as JSON) from a device
2. Fetch the last stored device configuration data (as formatted HTML) 

In [1]:
import json

In [2]:
configJsonFileName = 'dconfig.json'

In [3]:
'''
    Python WSGI sample application to 
    store some configuration data from
    POST requests and allow fetching the 
    last configuration data through
    GET request as formatted HTML
    
    Sample configuration JSON:
    {
        "firmware": "any_firmware_1.0.1",
        "time": "2018-10-07T20:20:08.775393",
        "state": "ON",
        "username": "pranav.s",
        "uptime": 1000
    }
'''
def application(environ, startResponse):
    # path of the requested resource
    path = environ['PATH_INFO']
    # for our scenario we'll only consider two paths valid
    status = '200 OK'
    
    if path == '/store':
        contentLength = int(environ['CONTENT_LENGTH'])
        postInputAsFile = environ['wsgi.input']
        rawPostBody = postInputAsFile.read(contentLength)
        body = rawPostBody.decode('utf-8')
        saveConfig(body, configJsonFileName)
        body = 'Saved Configuration Data'
    elif path == '/fetch':
        configJson = loadConfig(configJsonFileName)
        body = generatePage(configJson)
    else:
        status = '404 Not Found'
        body = 'Resource Not Found'
        
    headers = [
        ('Content-Type', 'text/html'),
        ('Content-Length', str(len(body)))
    ]
    startResponse(status, headers)
    return [body.encode('utf-8')]

In [4]:
def saveConfig(configJson, fileName):
    with open(fileName, 'w') as jsonFile:
        print(configJson, end = '', file = jsonFile)
def loadConfig(fileName):
    with open(fileName, 'r') as jsonFile:
        configJson = jsonFile.read()
    return configJson

In [5]:
def generatePage(configJson):
    configData = json.loads(configJson)
    # expects all attributes to exist (firmware, time, state, error, username, uptime)
    style = '''
        <style>
            th {
                border-bottom: 2px solid black;
                text-transform: uppercase;
                width: 200px;
                padding: 10px;
            }
            td {
                width: 200px;
                padding: 10px;
            }
            tr:nth-child(even) {
                background-color: gray;
                color: white;
            }
            tr:nth-child(odd) {
                width: 40px;
                background-color: lightgrey;
            }
        </style>
    '''
    html = '''
        <h1>Last Configuration Data</h1>
        <table>
            <tr> <th> Attribute </th> <th> Value </th> </tr>
            <tr> <td> Firmware Id </td> <td> {firmware} </td> </tr>
            <tr> <td> Date and Time </td> <td> {time} </td> </tr>
            <tr> <td> Current State </td> <td> {state} </td> </tr>
            <tr> <td> Username </td> <td> {username} </td> </tr>
            <tr> <td> Uptime </td> <td> {uptime} </td> </tr>
        </table>
    '''
    
    return style + html.format(firmware = configData['firmware'],
                       time = configData['time'],
                       state = configData['state'],
                       username = configData['username'],
                       uptime = configData['uptime']
                      )

In [6]:
port = 8080
host = ''
from wsgiref.simple_server import make_server
try:
    httpd = make_server(host, port, application)
    print('starting server on port', port)
    httpd.serve_forever()
except KeyboardInterrupt:
    print('stopping server')

starting server on port 8080


127.0.0.1 - - [07/Oct/2018 23:00:52] "POST /store HTTP/1.1" 200 24
127.0.0.1 - - [07/Oct/2018 23:00:52] "GET /fetch HTTP/1.1" 200 1025
127.0.0.1 - - [07/Oct/2018 23:01:01] "POST /store HTTP/1.1" 200 24
127.0.0.1 - - [07/Oct/2018 23:01:01] "GET /fetch HTTP/1.1" 200 1025
127.0.0.1 - - [07/Oct/2018 23:01:06] "GET /fetch HTTP/1.1" 200 1025
127.0.0.1 - - [07/Oct/2018 23:01:18] "GET /fetch HTTP/1.1" 200 1025
127.0.0.1 - - [07/Oct/2018 23:01:27] "GET /fetch HTTP/1.1" 200 1025
127.0.0.1 - - [07/Oct/2018 23:01:36] "GET /fetch HTTP/1.1" 200 1025
127.0.0.1 - - [07/Oct/2018 23:01:41] "GET /fetch HTTP/1.1" 200 1025


stopping server


@selfLink: https://github.com/swghosh/UG-Courses/blob/master/Others/MMIPart1.ipynb